# Householder-Transformation schrittweise

In [80]:
import numpy as np
from numpy.linalg import norm
from numpy.linalg import cond


Die allgemeine Schreibweise der Householder-Transformation für einen beliebigen Vektor $w$ ist gegeben durch
\begin{equation} H(w) = \text{id} - 2\,\frac{w\cdot w^T}{\langle w, w\rangle}
\end{equation}
wobei $w\cdot w^T$ das Kroneckerprodukt
$$w\cdot w^T = (w_i\,w_j)_{i,j=1\ldots m}\in\mathbb{R}^{m\times m}$$
sei.

In [81]:
# Selber implementieren
def HouseholderTransformation(w):
    size = w.shape[0]

    id = np.eye(size)
    kronecker = np.outer(w,w)
    dot = np.dot(w,w)


    return id-2*(kronecker/dot)

Gesucht ist der geeignete Normalenvektor so, dass der gespiegelte Spaltenvektor auf die $e_1$ Achse zu liegen kommt. Sei mit $y$ der Spaltenvektor bezeichnet, so kann man zeigen (siehe Skript), dass der Vektor
\begin{equation}
w = y \pm \|y\|_2 e_1
\end{equation}
die gewünschte Eigenschaft hat. Um **Auslöschung** in der Berechnung von $w$ zu vermeiden, wählt man
\begin{equation}
w = y + \text{sign}(y_1) \|y\|_2 e_1
\end{equation}
mit
\begin{equation}
\text{sign}(s) = \begin{cases} 1 & \quad \text{für} s \ge 0\\ -1 & \quad \text{sonst}.\end{cases}
\end{equation}

In [82]:
def mysign(x): # numpy sign liefert 0 für 0
    if x >= 0:
        return 1
    else:
        return -1

Funktion für den n-dimensionalen Einheitsvektor

In [83]:
def e(n):
    return np.array([1]+[0 for k in range(n-1)])

Mit Hilfe der Householder-Transformation soll nun die Matrix $A$ in eine orthogonale Matrix $Q$ und reguläre obere Dreiecksmatrix $R$ zerlegt werden. Im Beispiel wählen wir eine zufällige Matrix $A \in \mathbb{R}^{10\times5}$.

In [84]:
A = np.array([[-1,  7, -8, -9,  6],
       [-6, -8,  0,  3,  8],
       [-4, -2,  8,  0, -2],
       [-1, -9,  4, -8,  2],
       [-3, -5, -5,  7, -4],
       [-7, -4,  7, -1,  5],
       [-9, -7,  6, -5, -8],
       [-4, -3, -5,  3, -6],
       [ 5,  7,  5, -4, -5],
       [ 4, -6, -8, -2, -5]],dtype=float)
m,n = A.shape

### 1. Spalte

In [85]:
k = 0

Die Hyperebene ist definiert durch

In [86]:
w = A[k:,k]+np.linalg.norm(A[k:,k], axis=0)*e(m-k)*mysign(A[k,k])


Für die Householder-Transformationsmatrix angewand auf $A$ erhalten wir

In [87]:
Q1 = HouseholderTransformation(w)
A1 = Q1@A

In der ersten Spalte der Zwischenmatrix $A_1$ stehen nun abgesehen vom ersten Eintrag Nullen:

In [88]:
print(np.round(A1,4))

[[ 15.8114  11.8269  -6.5143  -0.6325  -1.2649]
 [ -0.      -6.2773   0.5303   5.9864   5.4071]
 [ -0.      -0.8515   8.3535   1.9909  -3.7286]
 [ -0.      -8.7129   4.0884  -7.5023   1.5679]
 [ -0.      -4.1386  -4.7349   8.4932  -5.2964]
 [ -0.      -1.9901   7.6186   2.4841   1.975 ]
 [ -0.      -4.4159   6.7954  -0.5204 -11.8893]
 [ -0.      -1.8515  -4.6465   4.9909  -7.7286]
 [  0.       5.5644   4.5581  -6.4887  -2.8393]
 [  0.      -7.1485  -8.3535  -3.9909  -3.2714]]


### 2. Spalte 

In [89]:
k = 1

Die Hyperebene ist definiert durch

In [90]:
w = A1[k:,k]+np.linalg.norm(A1[k:,k], axis=0)*e(m-k)*mysign(A1[k,k])

wobei nun das letzte Resultat $A_1$ benutzt wird. Die Householder-Transformationsmatrix wird nun nur auf die Submatrix von $A_1$ angewand und in der Submatrix von $A_1$ wiederum gespeichert:

In [91]:
Q2 = HouseholderTransformation(w)
A1[k:,k:] = Q2@A1[k:,k:]

Die Dimension der zweiten Householder-Transformationsmatrix $Q_2$ ist

In [92]:
Q2.shape

(9, 9)

In dem ersten beiden Spalte der Zwischenmatrix $A_1$ steht:

In [93]:
print(np.round(A1,4))

[[ 15.8114  11.8269  -6.5143  -0.6325  -1.2649]
 [ -0.      15.5603   1.4167  -1.8329   3.0822]
 [ -0.       0.       8.3881   1.686   -3.8192]
 [ -0.       0.       4.442  -10.622    0.6402]
 [ -0.       0.      -4.5669   7.0113  -5.737 ]
 [ -0.      -0.       7.6994   1.7715   1.7631]
 [ -0.       0.       6.9746  -2.1016 -12.3594]
 [ -0.      -0.      -4.5713   4.328   -7.9257]
 [  0.      -0.       4.3323  -4.4962  -2.2469]
 [  0.       0.      -8.0633  -6.5505  -4.0325]]


### 3. - 5. Spalte 

Wir automatisieren nun den Prozess und überschreiben die Submatrizen der Matrix $A_1$ sukzessive:

In [94]:
for k in range(2,n):
    print('Spalte '+str(k+1))
    w = A1[k:,k]+np.linalg.norm(A1[k:,k], axis=0)*e(m-k)*mysign(A1[k,k])
    Qk = HouseholderTransformation(w)
    A1[k:,k:] = Qk@A1[k:,k:]
    print(np.round(A1,4))

Spalte 3
[[ 15.8114  11.8269  -6.5143  -0.6325  -1.2649]
 [ -0.      15.5603   1.4167  -1.8329   3.0822]
 [ -0.       0.     -17.9877   2.92     0.9232]
 [ -0.       0.       0.     -10.4142   1.4389]
 [ -0.       0.       0.       6.7976  -6.5582]
 [ -0.      -0.      -0.       2.1317   3.1475]
 [ -0.       0.       0.      -1.7753 -11.1054]
 [ -0.      -0.       0.       4.1141  -8.7476]
 [  0.      -0.      -0.      -4.2936  -1.4679]
 [  0.       0.       0.      -6.9278  -5.4823]]
Spalte 4
[[ 15.8114  11.8269  -6.5143  -0.6325  -1.2649]
 [ -0.      15.5603   1.4167  -1.8329   3.0822]
 [ -0.       0.     -17.9877   2.92     0.9232]
 [ -0.       0.       0.      15.6753  -1.5851]
 [ -0.       0.       0.      -0.      -5.7703]
 [ -0.      -0.      -0.      -0.       3.3946]
 [ -0.       0.       0.       0.     -11.3111]
 [ -0.      -0.       0.       0.      -8.2708]
 [  0.      -0.      -0.       0.      -1.9656]
 [  0.       0.       0.       0.      -6.2853]]
Spalte 5
[[ 15.8114 

### Q, R berechnen

- Berechnen sie abschliessend $Q,R$ so, dass $Q\cdot R = A$ gilt.
- Vergleichen Sie Ihr Resultat mit der Funktion von NumPy.

In [95]:
cond(A)

2.39058310879836

In [96]:
A1 = A.copy()
QT = np.eye(m)
for k in range(0,n):
    w = A1[k:,k]+np.linalg.norm(A1[k:,k], axis=0)*e(m-k)*mysign(A1[k,k])
    Qk = HouseholderTransformation(w)
    A1[k:,k:] = Qk@A1[k:,k:]
    Q_trans = np.eye(m)
    Q_trans[k:,k:] = Qk
    QT = Q_trans@QT


In [104]:

Q = QT.T
R = A1

print(cond(R))
print(Q.shape)
print(R.shape)

#print(np.round(A-Q@R,4))
norm(A-Q@R)

2.390583108798361
(10, 10)
(10, 5)


1.4334856879248995e-14

In [103]:
Q,R = np.linalg.qr(A)
print(cond(R))

norm(A-Q@R)

2.39058310879836


7.954902093447287e-15

In [99]:
Q.shape

(10, 5)

In [100]:
R.shape

(5, 5)

Frage: Warum reicht diese reduzierte $Q$ und $R$ Matrix?

## answer
while the householder method can be more robust, we can see that in this case the condition number from the R in the householder matrix is not exactly the same as the one for A, but with the python implementation it is exactly the same condition number.

the python implementation is a bit more precise, but the householder is not that bad in this situation overall.

in this case, doing the householder takes more computing power, since we are operating with a matrix that is double in size line-wise, while the python implementation takes less operations to do a better job, thus it is, in this case, the better method.
